In [1]:
# Package import
from mri.operators import NonCartesianFFT, WaveletUD2, WaveletN , gradient
from mri.operators.utils import convert_mask_to_locations, convert_locations_to_mask
from mri.reconstructors import SingleChannelReconstructor
import pysap
from pysap.data import get_sample_data
import time

/home/guillaume/temp/pysap-mri/mri/operators/fourier/cartesian.py:33: UserWarning: pynufft python package has not been found. If needed use the master release. Till then you cannot use NUFFT on GPU
  warnings.warn("pynufft python package has not been found. If needed use "
/home/guillaume/temp/pysap-mri/mri/operators/fourier/non_cartesian.py:33: UserWarning: pynufft python package has not been found. If needed use the master release. Till then you cannot use NUFFT on GPU
  warnings.warn("pynufft python package has not been found. If needed use "



                 .|'''|       /.\      '||'''|,
                 ||          // \\      ||   ||
'||''|, '||  ||` `|'''|,    //...\\     ||...|'
 ||  ||  `|..||   .   ||   //     \\    ||
 ||..|'      ||   |...|' .//       \\. .||
 ||       ,  |'
.||        ''

Package version: 0.0.3

License: CeCILL-B

Authors: 

Antoine Grigis <antoine.grigis@cea.fr>
Samuel Farrens <samuel.farrens@cea.fr>
Jean-Luc Starck <jl.stark@cea.fr>
Philippe Ciuciu <philippe.ciuciu@cea.fr>

Dependencies: 

scipy          : >=1.3.0   - required | 1.3.0     installed
numpy          : >=1.16.4  - required | 1.16.4    installed
matplotlib     : >=3.0.0   - required | 3.1.1     installed
astropy        : >=3.0.0   - required | 3.2.1     installed
nibabel        : >=2.3.2   - required | 2.5.1     installed
pyqtgraph      : >=0.10.0  - required | 0.10.0    installed
progressbar2   : >=3.34.3  - required | ?         installed
modopt         : >=1.4.0   - required | 1.4.1     installed
scikit-learn   : >=0.19.1  - requi

In [2]:
# Third party import
from modopt.math.metrics import ssim
from modopt.opt.linear import Identity
from modopt.opt.proximity import SparseThreshold,Ridge,ProximityParent,ProximityCombo

import numpy as np
import matplotlib.pyplot as plt

## Data and operators

In [6]:
u0 = get_sample_data('2d-mri')
u0_mat = u0.data

n=u0_mat.shape[0]
x = np.linspace(0,n,n,endpoint=False)/n-0.5
y = np.linspace(0,n,n,endpoint=False)/n-0.5
X,Y = np.meshgrid(x,y)
kspace_loc = np.stack([X.flatten('F'),Y.flatten('F')],axis=1)

m = convert_locations_to_mask(kspace_loc,u0.shape)
fourier_op = NonCartesianFFT(samples=kspace_loc, shape=u0.shape,implementation='cpu')

# Adding noise
(n1,n2) = u0.shape
s = 1e-6
# Computing y with non-cartesian Fourier transform
fourier_op = NonCartesianFFT(samples=kspace_loc, shape=u0.shape,implementation='cpu')
y = (1+0j)*fourier_op.op(np.real(u0.data))+s*np.random.randn(2*n*n).view(np.complex128)
u0_mat = np.asarray(u0)

In [7]:
#Subsampling: From pysap radial mask 
radial_mask = get_sample_data("mri-radial-samples")
radial_kspace_loc = radial_mask.data

p = np.zeros((n*n,))

ind1 = ((n-1)*(radial_kspace_loc[:,0]+0.5)).astype(int)
ind2 = ((n-1)*(radial_kspace_loc[:,1]+0.5)).astype(int)
p[n*ind1+ind2]=1

new_loc = kspace_loc[np.where(p)]
#pysap.Image(data = convert_locations_to_mask(np.array(new_loc),u0.shape)).show()

#Wavelet
linear_op = WaveletN(wavelet_name="db8",nb_scale=4,padding_mode = "periodization")

In [8]:
#ifft on noised data = base result
uk = fourier_op.adj_op(p*y)
print(ssim(uk,u0_mat))
pysap.Image(data = np.abs(uk)).show()

0.6655194518060769


## CondatVu
### Regularizers
See modopt/opt/agorithms.py: Condat uses Moreau identity for prox_dual: we need to define the proximal operator and not the proximal dual.

In [9]:
class RegularizerG(ProximityParent):
    r"""negative values penalisation and L2 norm

    Parameters
    ----------
    linear : class
        Linear operator class
    weights : np.ndarray
        Input array of weights
    """

    def __init__(self, zeta, eps):
        self.weights = [eps,zeta]
        self.eps = eps
        self.zeta=zeta
        self.op = self._op_method
        self.cost = self._cost_method

        
    def _op_method(self, data, extra_factor=1.0):  
        thresh = self.eps*extra_factor
        
        return np.where(data>=0,data/(1+thresh),
                 data/(0.5+np.sqrt(0.25-3*self.zeta*extra_factor*data/(1+thresh)**2)))

    
    def _cost_method(self, *args, **kwargs):
        
        cost_val = np.abs(self.zeta*np.sum(np.where(args[0]<=0,-args[0]**3,0))
                           +self.eps*np.linalg.norm(args[0])**2/2)

        if 'verbose' in kwargs and kwargs['verbose']:
            print(' - G (X):', cost_val)

        return cost_val

In [10]:
class RegularizerF2(ProximityParent):
    r"""C2 approximation of an L1 norm

    Parameters
    ----------
    linear : class
        Linear operator class
    weights : np.ndarray
        Input array of weights
    """

    def __init__(self, linear, gamma, pn1):
        self.linear = linear
        self.weights = [gamma,pn1]
        self.gamma = gamma
        self.pn1=pn1
        self.op = self._op_method
        self.cost = self._cost_method
    
    def rho(self, x):
        m = np.abs(x)
        return np.where(m<self.gamma,-m**3/3/self.gamma**2+m**2/self.gamma,m-self.gamma/3)    
    
    def prox_J(self,x,c,gamma):
        return np.where(np.abs(x)>gamma+c,x*(1-c/np.abs(x)),
                    gamma*x/(c+0.5*gamma+np.sqrt((c+0.5*gamma)**2-c*np.abs(x))))

        
    def _op_method(self, data, extra_factor=1.0):  
        return self.prox_J(data,self.pn1*extra_factor,self.gamma)
        
    
    def _cost_method(self, *args, **kwargs):
        
        cost_val = self.pn1*np.sum(self.rho(self.linear.op(args[0])))

        if 'verbose' in kwargs and kwargs['verbose']:
            print(' - F2 (X):', cost_val)

        return cost_val

In [11]:
zeta,epsilon,gamma,pn1 = 1e15,2e-1,1e-10,2e-6
regularizer_G = RegularizerG(zeta,epsilon)
regularizer_F2 = RegularizerF2(linear_op,gamma,pn1)

In [12]:
print(regularizer_G.cost(u0_mat),regularizer_F2.cost(u0_mat))

3.8146968108776627e-07 2.296999543905258e-07


### Reconstruction

In [13]:
from modopt.opt.algorithms import Condat
from mri.operators import GradAnalysis
from modopt.opt.gradient import GradParent,GradBasic

Definition of gradient op: see https://cea-cosmic.github.io/ModOpt/_modules/modopt/opt/gradient.html

In [23]:
real_fourier_op = NonCartesianFFT(samples=kspace_loc, shape=u0.shape,implementation='cpu')
real_fourier_op.op = lambda x:fourier_op.op(np.real(x))
real_fourier_op.adj_op = lambda x:np.real(fourier_op.adj_op(x))
gradient_op = GradBasic(p*y,real_fourier_op.op,real_fourier_op.adj_op)

In [24]:
#Initialisation
gradient_op.fourier_op = real_fourier_op
norm = linear_op.l2norm(gradient_op.fourier_op.shape)
x_init = fourier_op.adj_op(p*y)

In [25]:
eta = pn1*5/2/gamma
mu = 2*np.sqrt(epsilon/2/eta)
theta = 1/(1+mu)

tau = mu/2/epsilon
sigma = mu*eta/2

In [26]:
opt = Condat(x=x_init,
            y=p*y,
            grad = gradient_op,
            prox = regularizer_G,
            prox_dual = regularizer_F2,
            linear = linear_op,
            sigma=sigma,
            tau=tau,
            sigma_update=None,
            tau_update=None,
            max_iter=150,
            auto_iterate = False)

In [27]:
opt.iterate(max_iter = 200)

/home/guillaume/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in sqrt
N/A% (0 of 200) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--

 - ITERATION: 1
 - DATA FIDELITY (X): 4.098140718492653e-08
 - G (X): 0.3905602661471222
 - F2 (X): 3.0032252455409367e-07
 - COST: 0.390560607451054



N/A% (0 of 200) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--

 - ITERATION: 2
 - DATA FIDELITY (X): 7.575442649225971e-08


  0% (1 of 200) |                        | Elapsed Time: 0:00:01 ETA:   0:04:04

...

 - G (X): 1.7178528231715404e-10
 - F2 (X): 2.0870408094771523e-09


 99% (198 of 200) |##################### | Elapsed Time: 0:02:04 ETA:   0:00:01

 - COST: 1.7981881284313485e-06

 - ITERATION: 200
 - DATA FIDELITY (X): 1.7959293082748173e-06


100% (200 of 200) |######################| Elapsed Time: 0:02:05 Time:  0:02:05


 - G (X): 1.7178525319023797e-10
 - F2 (X): 2.0870167989549544e-09
 - COST: 1.7981881103269626e-06

 - CONVERGENCE TEST - 
 - CHANGE IN COST: 1.5006528540300093e-07



In [28]:
uk = opt.x_final
print(ssim(np.real(uk),u0_mat))
pysap.Image(data = np.real(uk)).show()

0.8412949490076079


In [20]:
print(regularizer_G.cost(uk),regularizer_F2.cost(uk))

4.0236278357540493e-10 3.193882683645833e-09
